## 1. Text Extraction
Here from the given pdf survey file we will extract the texts and save it as text file for individual page of the survey report.

In [4]:
import re
def sorted_nicely( l ):
    convert = lambda text: int(text) if text.isdigit() else text
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)', key)]
    return sorted(l, key = alphanum_key)

In [115]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.converter import XMLConverter, HTMLConverter, TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfparser import PDFParser
import io
import os

#saving all the pages on different txt file
#can initiate text series
with open('SOUTH_BUO_ReportV2.pdf', 'rb') as fp:
    rsrcmgr = PDFResourceManager()
    retstr = io.StringIO()
    print(type(retstr))
    codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, codec=codec, laparams=laparams)
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    page_no = 1
    for pageNumber, page in enumerate(PDFPage.get_pages(fp)):
        if pageNumber == page_no-1:
            interpreter.process_page(page)
            data = retstr.getvalue()
            
            with open(os.path.join('text_files/', f'pdf_page_{page_no}.txt'), 'wb') as file:
                file.write(data.encode('utf-8'))
            data = ''
            retstr.truncate(0)
            retstr.seek(0)

        page_no += 1

<class '_io.StringIO'>


In [5]:
text_data_dict = {}
text_data_list = []
for file in sorted_nicely(os.listdir('text_files')):
    file_path = 'text_files/'+file
    with open(file_path, 'r', encoding='utf-8') as fp:
        string = fp.read()
        text_data_dict[file] = string
        text_data_list.append(string)

In [118]:
#view the data in the given position of the list
(text_data_list[1])

'XYZ\nXYZ SOUTH BUOY ROV UWILD INSPECTION 2018\n\nCONFIDENTIAL DO NOT DISTRIBUTE\n\nREVISION RECORD / HOLD RECORD SHEET\n\nREVISION RECORD\n\nRev\n\nRevision Description\n\nIssue Date\n\nRevision Amendment Details\n\nA1\nA2\n\nIssued for Approval\nIssued for Approval\n\nInitial Release\nAdded appendix for centre well top side photos\n\nHOLD RECORD\n\nHold Ref\n\n<HOLD01>\n<HOLD02>\n<HOLD03>\n<HOLD04>\n<HOLD05>\n\nDescription / Reason for Hold\n\nDocument Number: XYZ-YZH-OII-REP-0506, Rev A2\n\nPage 2 of 214\n\n\x0c'

## 2. Tabular Extraction
After getting the data from the pdf as text now we will perform tabular extraction from the given survey report. 

In [6]:
def check_for_line_in_file(file_name, string):
    with open(file_name, 'r', encoding='utf-8') as file:
        #for line_index, line in enumerate(file.read().splitlines()):
        for line_index, line in enumerate(file):
            if line.lower().startswith(string.lower()):
                return line_index
        return -1

In [7]:
import os
def get_file_name_with_the_string(path, line_to_check):
    for file in sorted_nicely(os.listdir(path)):
        path_and_file = path + '/' + file
        if check_for_line_in_file(path_and_file, line_to_check) != -1:
            return file

In [8]:
def last_integer_of_string(str):
    result = ""
    for i in range(len(str),0,-1):
        if str[i-1].isdigit():
            result = str[i-1]+result
        else:
            if result.isdigit():
                break
    return result

In [ ]:
#saving the tables as pickle(csv) files
import camelot


In [6]:
import os
import pandas as pd
import pickle

#dumping the files in the pickle
#for file in os.listdir('survey_1_50'):
   # d_f = pd.read_csv('survey_1_50/'+file)
    #with open("tables/" + file , "wb") as file1:  
       # pickle.dump(d_f, file1)

In [22]:
import os
import pandas as pd
data_frames_dict = {}
#data_frames_list = []
for file in sorted_nicely(os.listdir('tables')):
    with open('tables/'+ file, "rb") as table_file:
        read_df =  pd.read_csv(table_file)
        data_frames_dict[file.replace('.csv','')] = read_df
        #data_frames_list.append(table3)

In [9]:
class Table_in_report:
    def __init__(self, name):
        self.name = name
        self.page_number = -1
        self.data_frame = []

In [23]:
for key_idx, key in enumerate(data_frames_dict):
    if key_idx <5:
        print(key)

survey_page_1_table_1
survey_page_2_table_1
survey_page_2_table_2
survey_page_17_table_1
survey_page_19_table_1


Finding the name of the tables from the tables index in the survey file

In [11]:
file_with_line_given = get_file_name_with_the_string('text_files','TABLE OF TABLES')
file_with_table_of_tables = 'text_files/'+file_with_line_given

tables_from_report_index_without_df = []
with open(file_with_table_of_tables, 'r', encoding='utf-8') as file:
    table_count = 1  
    for line in file:
        line = line.lower()
        table_number = 'table '+str(table_count)
        if line.startswith(table_number):
            table_name = line.replace(table_number,'')
            table_count+=1
            page_number_position = last_integer_of_string(line)
            for s in ['.',page_number_position]:
                table_name = table_name.replace(s,'')
            #print(table_name)
            new_table_without_df = Table_in_report(table_name)
            new_table_without_df.page_number = int(page_number_position)-1
            #print(new_table_without_df.page_number)
            tables_from_report_index_without_df.append(new_table_without_df)
            #print(tables_from_report_index_without_df[table_count-2].page_number)

## 3.Image Extraction

## 4. Creating a class page with (Text, Table, Image) 

In [12]:
class Extracted_Page:
    def __init__(self, name):
        self.name = name
        self.text = ''
        self.tables = []
        self.Images = []

In [24]:

#for table in tables_from_report_index_without_df:
    #print(table.page_number)


#### Collect text, tables and Image for each extracted page

In [44]:
#creating the dictionary of the extracted page
extracted_Pages = {}
#each key gives each page
#also can be taken from the text directory
table_index_in_named_list = 0;
total_table_countFrom_index = len(tables_from_report_index_without_df)
table_name = 'None'
for page_index, key in enumerate(text_data_dict, start=1):
    new_name = key
    for s in ['pdf_','.txt']:
        new_name = new_name.replace(s,'') 
    new_page = Extracted_Page(new_name)    #forming Extracted_page for all the pages
    new_page.text = text_data_dict[key]  #setting the text to the new page
    
    #find the table in the tables directory related to the page
    #page_number_txt = key.replace('pdf_page_','')
    #page_number = page_number_txt.replace('.txt','')
    df_key_to_look_after = 'survey_page_'+str(page_index)+'_table_'
    data_frame_array_in_the_page = [data_frames_dict[df_key] for df_key in data_frames_dict if (df_key.startswith(df_key_to_look_after))]
    if len(data_frame_array_in_the_page) == 0:
        extracted_Pages[new_name] = new_page
        continue
    #print('for page',page_index,'found data frames are:', len(data_frame_array_in_the_page))
    #instead of data_frame_storing table with name, df and page_number
    tables_array = []
    #print(page_index)

    # get the name from the tables_dictionary extracted from the file contents
    if table_index_in_named_list <= total_table_countFrom_index-1 : 
        for d_f in data_frame_array_in_the_page: 
            table_without_df = tables_from_report_index_without_df[table_index_in_named_list]
            new_table = Table_in_report(table_without_df.name)
            indexed_page_in_table_sheet = table_without_df.page_number
            
            #print('table_page_numer:', table_without_df.page_number)
            #print('page_numer:', page_index)
            #print('table_number',table_index_in_named_list)
            
            if indexed_page_in_table_sheet == page_index:
                table_name = new_table.name
                #print(table_name)
                table_index_in_named_list += 1;
                
            new_table.name = table_name
            new_table.data_frame = d_f
            new_table.page_number = page_index
            tables_array.append(new_table)
        new_page.tables = tables_array #setting the tables to the page
        #print(len(new_page.tables))
    extracted_Pages[new_name] = new_page

In [47]:
import pickle
with open('extracted_Pages.pickle' , 'wb') as handle:
    pickle.dump(extracted_Pages, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [49]:
with open('extracted_Pages.pickle', 'rb') as handle:
    extracted_Pages1 = pickle.load(handle)

In [50]:
extracted_Pages1['page_19'].tables[0].name

' scope of work completion \n'

In [45]:
#extracted_Pages['page_19'].text

In [ ]:
extracted_Pages = {}
for key in text_data_dict:
    page = extracted_Page(key)
    page.text = text_data_dict[key]
    page_number_txt = key.replace('pdf_page_','')
    page_number = page_number_txt.replace('.txt','')
    table_key_to_look_after = 'page_'+page_number+'_table_'
    page.tables = [tables[table_key] for table_key in tables if (table_key.startswith(table_key_to_look_after))]
    extracted_Pages[key] = page

## 5. Cleaning and re-arranging the data

### i. Cleaning Text data

### ii. Tabular data arranging 

In [44]:
#creating transpose to remove extra column created
import numpy as np
def remove_unwanted_rows_colums(df):
    df = df.dropna(axis ='columns', thresh = 1)
    df = df.dropna(axis='index', thresh = 1 )
    return df

In [8]:
marine_ass_growth_df = data_frames[21]

In [99]:
marine_ass_growth_df = remove_unwanted_rows_colums(marine_ass_growth_df)

In [149]:
def fill_with_header_for_empty_space(df):
    string_replacing_with = 'NaN'
    for col in df.columns:
        print(string_replacing_with)
        if col[:3] == 'Unn':
            if string_replacing_with != 'NaN':
                df = df.rename(columns={col:string_replacing_with})
                #print(df)
            else:
                continue
        else:
            string_replacing_with = col
    
    return df

In [111]:
#marine_ass_growth_df = fill_with_header_for_empty_space(marine_ass_growth_df)
#fill_with_header_for_empty_space(marine_ass_growth_df)

In [11]:
marine_ass_growth_df

,Marine Growth Assessment,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,Location,% Cover,NaN,"Max Thickness,\nmm",NaN,"Average thickness,\nmm",NaN,Morphology /\ntype of\nmarine growth,NaN
1,NaN,Soft,Hard,Soft,Hard,Soft,Hard,Soft,Hard
2,05-1,70,60,25,50,20,35,Hydroids\nSponges,Coral


In [2]:
def add_top_header_for_NAN_spaces(df,n_row):
    header_replacing_with = 'NaN'
    col_idx = -1
    row_length = len(df.index)
    horizontal_replacing_strings = ['NaN']*n_row
    for col in df.columns:
        col_idx +=1 
        #replacement of Unnamed for column name
        if col[:3] == 'Unn':
            if header_replacing_with != 'NaN':
                df = df.rename(columns={col:header_replacing_with})
                #print(df)
        else:
            header_replacing_with = col
        for j in range(n_row):
            cell_data = df.iloc[j,col_idx]
            #print(cell_data)
            if pd.isnull(df.iloc[j,col_idx]):
                if horizontal_replacing_strings[j] != 'NaN':
                    df.iat[j, col_idx] = horizontal_replacing_strings[j]
            else:
                horizontal_replacing_strings[j] = cell_data 
                #print(horizontal_replacing_strings)
        #replacing NAN with vertical replacement
        #if (col_idx < 3)
        
    return df

In [161]:
new_df 

,Marine Growth Assessment,Marine Growth Assessment,Marine Growth Assessment,Marine Growth Assessment,Marine Growth Assessment,Marine Growth Assessment,Marine Growth Assessment,Marine Growth Assessment,Marine Growth Assessment
0,Location,% Cover,% Cover,"Max Thickness,\nmm","Max Thickness,\nmm","Average thickness,\nmm","Average thickness,\nmm",Morphology /\ntype of\nmarine growth,Morphology /\ntype of\nmarine growth
1,NaN,Soft,Hard,Soft,Hard,Soft,Hard,Soft,Hard
2,05-1,70,60,25,50,20,35,Hydroids\nSponges,Coral


,Marine Growth Assessment,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,Location,Madhu,NaN,"Max Thickness,\nmm",NaN,"Average thickness,\nmm",NaN,Morphology /\ntype of\nmarine growth,NaN
1,NaN,Soft,Hard,Soft,Hard,Soft,Hard,Soft,Hard
2,05-1,70,60,25,50,20,35,Hydroids\nSponges,Coral


In [129]:
df100 =  pickle.load(open('tables/tables101.csv', "rb"))

In [130]:
df100

,Mooring\nLine,Anomaly,Type,Detail
0,1.0,None,NaN,NaN
1,2.0,None,NaN,NaN
2,3.0,None,NaN,NaN
3,4.0,YZH/2018OI3/02,AW,Connector tube anode is 50% depleted.
4,5.0,YZH/2018OI3/03,AW,Connector tube anode is 100% depleted.
5,NaN,NaN,CP,A low CP reading obtained on the connector tub...
6,6.0,YZH/2018OI3/06,AW,Connector tube anode is 100% depleted.
7,NaN,NaN,CP,A low CP reading obtained on the connector tub...
8,NaN,NaN,PD,A continuity strap on the connector body is se...
9,NaN,NaN,BM,The severed cable is fretting against the plat...


In [48]:
def add_side_header_for_NAN_spaces(df, n_col):
    horizontal_replacing_strings = ['NaN', 'NaN', 'NaN']
    for col in df.columns:
        col_idx +=1 
        #replacement of Unnamed for column name
        if col[:3] == 'Unn':
            if header_replacing_with != 'NaN':
                df = df.rename(columns={col:header_replacing_with})
                #print(df)
        else:
            header_replacing_with = col
        #row_idx = 0
        for j in range(3):
            cell_data = df.iloc[j,col_idx]
            #print(cell_data)
            if pd.isnull(df.iloc[j,col_idx]):
                if horizontal_replacing_strings[j] != 'NaN':
                    df.iat[j, col_idx] = horizontal_replacing_strings[j]
            else:
                horizontal_replacing_strings[j] = cell_data 
                #print(horizontal_replacing_strings)
        #replacing NAN with vertical replacement
        #if (col_idx < 3)
        
    return df

3

In [141]:
horizontal_replacing_strings = ['NaN']*3

In [142]:
horizontal_replacing_strings

['NaN', 'NaN', 'NaN']